In [1]:
import pennylane as qml
from typing import Callable, Sequence
from pennylane.tape import QuantumTape
from functools import partial


def encode():
    i = 0
    while i < 9:
        qml.CNOT([i, i + 3])
        qml.CNOT([i, i + 6])
        
        qml.Hadamard(i)
        qml.Hadamard(i + 3)
        qml.Hadamard(i + 6)
        
        qml.CNOT([i, i + 1])
        qml.CNOT([i + 3, i + 4])
        qml.CNOT([i + 6, i + 7])
        
        qml.CNOT([i, i + 2])
        qml.CNOT([i + 3, i + 5])
        qml.CNOT([i + 6, i + 8])
        qml.Barrier(range(0,9))
        i += 9
        
def decode_x():
    i = 0
    while i < 9:
        qml.CNOT([i, i + 1])
        qml.CNOT([i, i + 2])

        qml.CNOT([i + 3, i + 4])
        qml.CNOT([i + 3, i + 5])
        
        qml.CNOT([i + 6, i + 7])
        qml.CNOT([i + 6, i + 8])
        
        j = i + 1
        while j < i + 7:
            one = qml.measure(j)
            two = qml.measure(j + 1)
            if one and two:
                qml.PauliX(i)
                break
            j += 3
        i += 9

def decode_z():
    i = 0
    while i < 9:
        qml.Hadamard(i)
        qml.Hadamard(i + 3)
        qml.Hadamard(i + 6)
        
        qml.CNOT([i, i + 3])
        qml.CNOT([i, i + 6])
        
        one = qml.measure(i + 3)
        two = qml.measure(i + 6)
        if one and two:
            qml.PauliZ(0)
            
        i += 9
    
with qml.tape.QuantumTape() as q_e:
    encode()
    
with qml.tape.QuantumTape() as q_d_x:
    decode_x()
    
with qml.tape.QuantumTape() as q_d_z:
    decode_z()
    
tape_e = qml.tape.QuantumTape.from_queue(q_e)
tape_d_x = qml.tape.QuantumTape.from_queue(q_d_x)
tape_d_z = qml.tape.QuantumTape.from_queue(q_d_z)

encode_ops = list(tape_e)
decode_ops_x = list(tape_d_x)
decode_ops_z = list(tape_d_z)


In [7]:

@qml.transform
def encode_transform(tape: qml.tape.QuantumTape, lst)-> (Sequence[QuantumTape], Callable):
    new_ops = lst + tape.operations
    return [QuantumTape(new_ops, tape.measurements, tape.shots, tape.trainable_params)], lambda res: res[0]


def optimize(gates):
    a = qml.tape.QuantumTape(gates)

    [b], _ = qml.transforms.cancel_inverses(a)
    return b.circuit

@qml.transform
def decode_transform_z(tape: qml.tape.QuantumTape, lst, use_optimizer=True)-> (Sequence[QuantumTape], Callable):
    next_layers = []
    num = 1
    all_layers = tape.operations
    gates_to_apply = []
    new_ops = []
    i = 0
    while i < len(all_layers):
        if all_layers[i].name == "PauliZ":
            if not use_optimizer:
                new_ops.append(all_layers[i])
                i += 1
                for l in lst:
                    new_ops.append(l)
            else:
                next_layers = all_layers[i+1: i + 2 ]
                actual_gates_to_apply = [all_layers[i]] + lst + next_layers
                gates_to_apply = optimize(actual_gates_to_apply)
                i += num + 1
                new_ops.extend(gates_to_apply)
        else:
            new_ops.append(all_layers[i])
            i += 1

    return [QuantumTape(new_ops, tape.measurements, tape.shots, tape.trainable_params)], lambda res: res[0]


@qml.transform
def decode_transform_x(tape: qml.tape.QuantumTape, lst, use_optimizer=True)-> (Sequence[QuantumTape], Callable):
    next_layers = []
    num = 1
    all_layers = tape.operations
    gates_to_apply = []
    new_ops = []
    i = 0
    while i < len(all_layers):
        if all_layers[i].name == "Identity":
            if not use_optimizer:
                if all_layers[i].name != "Identity":
                    new_ops.append(all_layers[i])
                i += 1
                for l in lst:
                    new_ops.append(l)
            else:
                next_layers = all_layers[i+1: i + 2 ]
                # actual_gates_to_apply = [all_layers[i]] + lst + next_layers
                actual_gates_to_apply = lst + next_layers
                gates_to_apply = optimize(actual_gates_to_apply)
                i += num + 1
                new_ops.extend(gates_to_apply)
        else:
            new_ops.append(all_layers[i])
            i += 1

    return [QuantumTape(new_ops, tape.measurements, tape.shots, tape.trainable_params)], lambda res: res[0]

def logical_hadamard(start, end):
    for i in range(start + 1, end + 1):
        qml.CNOT([0, i])
    qml.Hadamard(start)
    for i in range(start + 1, end + 1):
        qml.CNOT([0, i])
        
def logical_x(start, end):
    print(start, end)
    for i in range(start, end):
        qml.PauliX(i)

num_logical_qubits = 2
physical_qubits_per_logical_qubit = 9
dev = qml.device("default.qubit", wires=num_logical_qubits * physical_qubits_per_logical_qubit)

@qml.qnode(dev)
@partial(decode_transform_x, lst=decode_ops_x, use_optimizer=False)
@partial(decode_transform_z, lst=decode_ops_z, use_optimizer=False)
@partial(encode_transform, lst=encode_ops)
def c():
    for i in range(0, num_logical_qubits):
        logical_hadamard(i * physical_qubits_per_logical_qubit, i * physical_qubits_per_logical_qubit + physical_qubits_per_logical_qubit - 1)
    
    #Error  
        qml.PauliX(i)
        qml.Identity(i)
    
        logical_x(i * physical_qubits_per_logical_qubit, i * physical_qubits_per_logical_qubit + physical_qubits_per_logical_qubit)
     
    return qml.expval(qml.PauliZ(0))

print(qml.draw(c)())

0 9
9 18
 0: ─╭●─╭●──H─╭●─╭●─╭||─╭●─╭●─╭●─╭●─╭●─╭●─╭●─╭●──H─╭●─╭●─╭●─╭●─╭●─╭●─╭●─╭●──X─╭●─╭●──X────X───────╭●
 1: ─│──│─────╰X─│──├||─╰X─│──│──│──│──│──│──│─────╰X─│──│──│──│──│──│──│─────╰X─│───┤↗├─────X────│─
 2: ─│──│────────╰X─├||────╰X─│──│──│──│──│──│────────╰X─│──│──│──│──│──│────────╰X──┤↗├────────X─│─
 3: ─╰X─│───H─╭●─╭●─├||───────╰X─│──│──│──│──│───────────╰X─│──│──│──│──│──╭●─╭●──X───────────────│─
 4: ────│─────╰X─│──├||──────────╰X─│──│──│──│──────────────╰X─│──│──│──│──╰X─│───X───────────────│─
 5: ────│────────╰X─├||─────────────╰X─│──│──│─────────────────╰X─│──│──│─────╰X──X───────────────│─
 6: ────╰X──H─╭●─╭●─├||────────────────╰X─│──│────────────────────╰X─│──│──╭●─╭●──X───────────────│─
 7: ──────────╰X─│──├||───────────────────╰X─│───────────────────────╰X─│──╰X─│───X───────────────│─
 8: ─────────────╰X─╰||──────────────────────╰X─────────────────────────╰X────╰X──X───────────────│─
 9: ──────────────────────────────────────────────────────────────────────────────

In [6]:
qml.specs(c)()

0 9
9 18


{'resources': Resources(num_wires=18, num_gates=82, gate_types=defaultdict(<class 'int'>, {'CNOT': 52, 'Hadamard': 5, 'Barrier': 1, 'PauliX': 20, 'Identity': 4}), gate_sizes=defaultdict(<class 'int'>, {2: 52, 1: 29, 9: 1}), depth=45, shots=Shots(total_shots=None, shot_vector=())),
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_trainable_params': 0,
 'num_device_wires': 18,
 'device_name': 'default.qubit',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

In [8]:
qml.specs(c)()

0 9
9 18


{'resources': Resources(num_wires=18, num_gates=84, gate_types=defaultdict(<class 'int'>, {'CNOT': 52, 'Hadamard': 5, 'Barrier': 1, 'PauliX': 22, 'Identity': 4}), gate_sizes=defaultdict(<class 'int'>, {2: 52, 1: 31, 9: 1}), depth=47, shots=Shots(total_shots=None, shot_vector=())),
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_trainable_params': 0,
 'num_device_wires': 18,
 'device_name': 'default.qubit',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}